In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='0,1'

In [4]:
import sys
import argparse
import numpy as np
import cv2

In [5]:
from multiprocessing.dummy import Pool as ThreadPool 

In [6]:
import tensorflow as tf

In [7]:
image = cv2.imread('104.bmp', -1)
image = cv2.resize(image, (256, 256), 0, 0, interpolation=cv2.INTER_LINEAR)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
np_image = np.asarray(image).astype(np.float)
np_image = np_image / 255.0

In [8]:
save_model_path = "saved_model.pb"

In [9]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

In [10]:
temp = []
temp.append(np_image)

In [ ]:
graph_def = tf.GraphDef()

In [ ]:
with tf.gfile.Open(save_model_path, 'rb') as f:
    graph_def.ParseFromString(f.read())

with tf.Graph().as_default() as graph:
    tf.import_graph_def(graph_def, name='')
    sess = tf.Session(graph=graph)
    input_tensor = graph.get_tensor_by_name('input_1:0')
    output_tensor = graph.get_tensor_by_name('conv2d_16/Sigmoid:0')

In [ ]:
# make the Pool of workers
pool = ThreadPool(2) 

In [ ]:
def run_test(device_name, count):
    for i in range(0,count):
        try:
            with tf.device(device_name):
                scores = []
                scores = sess.run(output_tensor, feed_dict={input_tensor: temp})
                if i%100==0:
                    print(device_name, scores.shape)
        except RuntimeError as e:
            print(e)    

In [ ]:
device_names = ['/device:GPU:0','/device:GPU:1']
counts = [50000, 50000]

In [ ]:
results = pool.starmap(run_test, zip(device_names, counts))
# close the pool and wait for the work to finish 
pool.close() 
pool.join() 

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for g in gpus:
    print("Name:",g.name," Type:",g.device_type)

In [ ]:
print(gpus)

In [ ]:
count = 10000
device_name = '/device:GPU:1'
for i in range(0,count):
    try:
        with tf.device(device_name):
            scores = []
            scores = sess.run(output_tensor, feed_dict={input_tensor: temp})
            if i%100==0:
                print(device_name, scores.shape)
    except RuntimeError as e:
        print(e)    

In [ ]:
count = 10000
device_name = '/device:GPU:0'
for i in range(0,count):
    try:
        with tf.device(device_name):
            scores = []
            scores = sess.run(output_tensor, feed_dict={input_tensor: temp})
            if i%100==0:
                print(device_name, scores.shape)
    except RuntimeError as e:
        print(e)    

In [22]:
from pprint import pprint
graphs = []
for g in range(1):
  graphs.append(tf.Graph())
  with graphs[g].as_default():
    graph_def = tf.GraphDef()
    with tf.gfile.Open(save_model_path, 'rb') as f:
        graph_def.ParseFromString(f.read())
    with tf.device('/gpu:' + str(g)):
        tf.import_graph_def(graph_def, name='')
        pprint(tf.contrib.graph_editor.get_tensors(tf.get_default_graph()))
        with tf.Graph().as_default() as graph:
            sess = tf.Session(graph=graph)
            input_tensor = graph.get_tensor_by_name('input_1:0')
#             output_tensor = graph.get_tensor_by_name('conv2d_16/Sigmoid:0')    


[<tf.Tensor 'input_1:0' shape=(?, 256, 256, 3) dtype=float32>,
 <tf.Tensor 'conv2d/kernel:0' shape=(3, 3, 3, 16) dtype=float32>,
 <tf.Tensor 'conv2d/bias:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'conv2d/Conv2D/ReadVariableOp:0' shape=(3, 3, 3, 16) dtype=float32>,
 <tf.Tensor 'conv2d/Conv2D:0' shape=(?, 256, 256, 16) dtype=float32>,
 <tf.Tensor 'conv2d/BiasAdd/ReadVariableOp:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'conv2d/BiasAdd:0' shape=(?, 256, 256, 16) dtype=float32>,
 <tf.Tensor 'batch_normalization/gamma:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'batch_normalization/beta:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'batch_normalization/moving_mean:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'batch_normalization/moving_variance:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'batch_normalization/ReadVariableOp:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'batch_normalization/ReadVariableOp_1:0' shape=(16,) dtype=float32>,
 <tf.Tensor 'batch_normalization/FusedBatchNorm/ReadVariableOp:0

KeyError: "The name 'input_1:0' refers to a Tensor which does not exist. The operation, 'input_1', does not exist in the graph."